In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv('../../results/result.csv')

repo_num = 1084
# merge_tool = "gitmerge-recursive-patience"
# merge_tool = "spork"
# merge_tool = "gitmerge-recursive-minimal"
# merge_tool = "gitmerge-resrepo_num = 3595
# merge_tool = "gitmerge-recursive-patience"
# merge_tool = "spork"
# merge_tool = "git merge-resolve"
# merge_tool = "intellimerge"
merge_tool = 'gitmerge-ort-ignorespace'
# merge_tool = 'gitmerge-ort'

repo_name = df.iloc[repo_num]["repo_name"]

script = "../scripts/merge_tools/" + merge_tool + ".sh"
from merge_tester import MERGE_STATE
df[merge_tool].apply(lambda x: MERGE_STATE[x])[0]
from validate_repos import clone_repo
repo = clone_repo(repo_name) # Return a Git-Python repo object
repo.remote().fetch()
repo.git.checkout(df.iloc[repo_num]["left"], force=True)
repo.submodule_update()
repo.git.checkout("-b", "TEMP_LEFT_BRANCH", force=True)
repo.git.checkout(df.iloc[repo_num]["right"], force=True)
repo.submodule_update()
repo.git.checkout("-b", "TEMP_RIGHT_BRANCH", force=True)
repo_path = repo.git.rev_parse("--show-toplevel")

import subprocess
import re
import os
result = subprocess.run([script, repo_path, "TEMP_LEFT_BRANCH", "TEMP_RIGHT_BRANCH"], stdout=subprocess.PIPE, text=True)
conflict_file_match = re.search(r'CONFLICT \(.+\): Merge conflict in (.+)', result.stdout)
conflicting_lines = []

if conflict_file_match:
    conflicting_file = conflict_file_match.group(1)
    full_path = os.path.join(repo_path, conflicting_file)

    # Open the conflicting file and print the base branch content for the conflicting lines
    with open(full_path, 'r') as file:
        file_content = file.read()

        # Split the conflicting file content into lines
        lines = file_content.split('\n')

        # Flag to indicate whether the current line is within a conflict marker
        in_conflict = False
        conflict_offset = -1

        df[merge_tool].apply(lambda x: MERGE_STATE[x])[0]
        from validate_repos import clone_repo2
        repo = clone_repo2(repo_name) # Return a Git-Python repo object
        repo.remote().fetch()
        repo.git.checkout(df.iloc[repo_num]["base"], force=True)
        repo.submodule_update()

        file_path2 = os.path.join('/home/benrr/miniconda3/envs/merge_1/AST-Merging-Evaluation/src/python/repos2', repo_name, conflicting_file)

        # Iterate through the lines and print the base branch content for conflicting lines
        base_lines = [">>>>>> BASE"]
        i = 0
        while (i < len(lines)):
            line = lines[i]
            if line.startswith("<<<<<<<"):
                conflict_offset += 1
                in_conflict = True
            elif line.startswith(">>>>>>>"):
                for j, line3 in enumerate(base_lines):
                    lines.insert(i + j + 1, line3)
                i += len(base_lines)
                conflict_offset += 2
            elif line.startswith("======="):
                in_conflict = False
            elif in_conflict:
                line_number = i - conflict_offset
                conflicting_lines.append(i - conflict_offset)
                try:
                    with open(file_path2, 'r') as file:
                        lines2 = file.readlines()
                        if 1 <= line_number <= len(lines2):
                            desired_line = lines2[line_number - 1]
                            base_lines.insert(0, desired_line)
                            print(f"Base {line_number}: {desired_line}")
                        else:
                            print(f"Error: Line number {line_number} is out of range for the file.")
                except FileNotFoundError:
                    print(f"Error: File '{file_path2}' not found.")
                except Exception as e:
                    print(f"An error occurred: {e}")
            i += 1

    with open(full_path, 'w') as file:
        file.write('\n'.join(lines))

    # Open the conflicting file with the default text editor
    subprocess.run(["xdg-open", full_path], check=True)
else:
    print("No conflicting file found in the output.")

print(conflicting_lines)

datumbox/datumbox-framework  : Cloning repo
Repository 'datumbox/datumbox-framework' cloned to path: repos/datumbox/datumbox-framework
datumbox/datumbox-framework  : Finished cloning


Switched to branch 'TEMP_LEFT_BRANCH'
fatal: not a tree object
/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/benrr/snap/code/common/.cache/gio-modules/libgiolibproxy.so


Conflicting line:         ClustererValidator.ValidationMetrics vm = instance.validate(validationData);

--- File Added in Both Parents ---

This file was added in both parents and is not present in the base branch.
Conflicting line:         MultinomialDPMM.ValidationMetrics vm = instance.validate(validationData);

--- File Added in Both Parents ---

This file was added in both parents and is not present in the base branch.

--- Rest of Conflicting File ---

/**
 * Copyright (C) 2013-2016 Vasilis Vryniotis <bbriniotis@datumbox.com>
 *
 * Licensed under the Apache License, Version 2.0 (the "License");
 * you may not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 *     http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing, software
 * distributed under the License is distributed on an "AS IS" BASIS,
 * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 * See the